In [1]:
%cd ../..

/Users/mlevydaniel/Desktop/modern-time-series-forecasting-with-python


In [2]:
import os
import time

import joblib
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"

import warnings
from functools import partial
from pathlib import Path

import humanize
from darts.metrics import mae, mase, mse
from sklearn.preprocessing import StandardScaler
from src.utils import plotting_utils
from src.utils.general import LogTime
from src.utils.ts_utils import darts_metrics_adapter, forecast_bias
from tqdm.autonotebook import tqdm
from IPython.display import display, HTML
# %load_ext autoreload
# %autoreload 2
np.random.seed(42)
import random

random.seed(42)
tqdm.pandas()

/Users/mlevydaniel/Desktop/modern-time-series-forecasting-with-python/src/utils/data_utils.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
os.makedirs("imgs/chapter_8", exist_ok=True)
preprocessed = Path("data/london_smart_meters/preprocessed")
output = Path("data/london_smart_meters/output")

In [5]:
def format_plot(fig, legends=None, xlabel="Time", ylabel="Value", title="", font_size=15):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title_text=title,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": 20},
        legend_title=None,
        legend=dict(
                font=dict(size=font_size),
                orientation="h",
                yanchor="bottom",
                y=0.98,
                xanchor="right",
                x=1,
            ),
            yaxis=dict(
                title_text=ylabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            ),
            xaxis=dict(
                title_text=xlabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            ),
    )
    return fig

# Reading the Test Predictions and Metrics

In [6]:
try:
    # Reading the missing value imputed and train test split data
    train_df = pd.read_parquet(
        preprocessed / "selected_blocks_train_missing_imputed_feature_engg.parquet"
    )
    train_df = train_df.loc[:, ["timestamp", "LCLid", "energy_consumption"]].set_index(
        ["timestamp", "LCLid"]
    )
    val_df = pd.read_parquet(
        preprocessed / "selected_blocks_val_missing_imputed_feature_engg.parquet"
    )
    val_df = val_df.loc[:, ["timestamp", "LCLid", "energy_consumption"]].set_index(
        ["timestamp", "LCLid"]
    )

    train_target = train_df.reset_index().set_index("timestamp")
    # Combine train and val into new train
    train_val_target = pd.concat([train_df, val_df]).reset_index().set_index("timestamp")

    del val_df, train_df
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Feature Engineering.ipynb in Chapter06
    </div>
    """))

In [14]:

pred_test_df = pd.read_pickle(output / "ml_single_step_prediction_test_df.pkl")
metrics_test_df = pd.read_pickle(output / "ml_single_step_metrics_test_df.pkl")
pred_auto_stat_test_df = pd.read_pickle(
    output / "ml_single_step_prediction_auto_stationary_test_df.pkl"
)
metrics_auto_stat_test_df = pd.read_pickle(
    output / "ml_single_step_metrics_auto_stationary_test_df.pkl"
)
agg_metrics_auto_stat_test_df = pd.read_pickle(
    output / "ml_single_step_aggregate_metrics_auto_stationary_test.pkl"
)
pred_baselines_test_df = pd.read_pickle(output / "baseline_test_prediction_df.pkl")
metrics_baselines_test_df = pd.read_pickle(output / "baseline_test_metrics_df.pkl")
agg_metrics_baselines_test_df = pd.read_pickle(
    output / "baseline_test_aggregate_metrics.pkl"
)


pred_val_df = pd.read_pickle(output / "ml_single_step_prediction_val_df.pkl")
metrics_val_df = pd.read_pickle(output / "ml_single_step_metrics_val_df.pkl")
pred_auto_stat_val_df = pd.read_pickle(
    output / "ml_single_step_prediction_auto_stationary_val_df.pkl"
)
metrics_auto_stat_val_df = pd.read_pickle(
    output / "ml_single_step_metrics_auto_stationary_val_df.pkl"
)
agg_metrics_auto_stat_val_df = pd.read_pickle(
    output / "ml_single_step_aggregate_metrics_auto_stationary_val.pkl"
)
pred_baselines_val_df = pd.read_pickle(output / "baseline_val_prediction_df.pkl")
metrics_baselines_val_df = pd.read_pickle(output / "baseline_val_metrics_df.pkl")
agg_metrics_baselines_val_df = pd.read_pickle(
    output / "baseline_val_aggregate_metrics.pkl"
)

In [16]:
pred_val_df = pd.concat([pred_val_df, pred_auto_stat_val_df, pred_baselines_val_df])
pred_val_df.index.name = "timestamp"

pred_wide_val = pd.pivot(
    pred_val_df.reset_index(),
    index=["LCLid", "timestamp"],
    columns="Algorithm",
    values="predictions",
)
pred_wide_val = pred_wide_val.join(
    pred_val_df.loc[
        pred_val_df.Algorithm == "Lasso Regression", ["LCLid", "energy_consumption"]
    ]
    .reset_index()
    .set_index(["LCLid", "timestamp"])
)
pred_wide_val.head()

FFT  Lasso Regression  \
LCLid     timestamp                                         
MAC000061 2014-01-01 00:00:00  0.111663          0.131014   
          2014-01-01 00:30:00  0.088207          0.114568   
          2014-01-01 01:00:00  0.070980          0.121935   
          2014-01-01 01:30:00  0.060706          0.112449   
          2014-01-01 02:00:00  0.056753          0.073503   

                               Lasso Regression_auto_stat  LightGBM  \
LCLid     timestamp                                                   
MAC000061 2014-01-01 00:00:00                    0.119521  0.113324   
          2014-01-01 00:30:00                    0.105024  0.092285   
          2014-01-01 01:00:00                    0.129562  0.098231   
          2014-01-01 01:30:00                    0.120926  0.080759   
          2014-01-01 02:00:00                    0.080298  0.059997   

                               LightGBM_auto_stat     Theta  \
LCLid     timestamp                                           
MAC000061 2014-01-01 00:00:00            0.086890  0.157924   
          2014-01-01 00:30:00            0.074833  0.150390   
          2014-01-01 01:00:00            0.072314  0.147509   
          2014-01-01 01:30:00            0.068520  0.146588   
          2014-01-01 02:00:00            0.054993  0.147599   

                               XGB Random Forest  XGB Random Forest_auto_stat  \
LCLid     timestamp                                                             
MAC000061 2014-01-01 00:00:00           0.119583                     0.088141   
          2014-01-01 00:30:00           0.105767                     0.068639   
          2014-01-01 01:00:00           0.107085                     0.072662   
          2014-01-01 01:30:00           0.102175                     0.054764   
          2014-01-01 02:00:00           0.075707                     0.048437   

                               energy_consumption  
LCLid     timestamp                                
MAC000061 2014-01-01 00:00:00               0.165  
          2014-01-01 00:30:00               0.167  
          2014-01-01 01:00:00               0.150  
          2014-01-01 01:30:00               0.091  
          2014-01-01 02:00:00               0.047

In [17]:
pred_test_df = pd.concat([pred_test_df, pred_auto_stat_test_df, pred_baselines_test_df])
pred_test_df.index.name = "timestamp"

pred_wide_test = pd.pivot(
    pred_test_df.reset_index(),
    index=["LCLid", "timestamp"],
    columns="Algorithm",
    values="predictions",
)
pred_wide_test = pred_wide_test.join(
    pred_test_df.loc[
        pred_test_df.Algorithm == "Lasso Regression", ["LCLid", "energy_consumption"]
    ]
    .reset_index()
    .set_index(["LCLid", "timestamp"])
)
pred_wide_test.head()

FFT  Lasso Regression  \
LCLid     timestamp                                         
MAC000061 2014-02-01 00:00:00  0.113955          0.057976   
          2014-02-01 00:30:00  0.087921          0.052869   
          2014-02-01 01:00:00  0.068605          0.055565   
          2014-02-01 01:30:00  0.056841          0.038992   
          2014-02-01 02:00:00  0.051993          0.026235   

                               Lasso Regression_auto_stat  LightGBM  \
LCLid     timestamp                                                   
MAC000061 2014-02-01 00:00:00                    0.069637  0.068784   
          2014-02-01 00:30:00                    0.056009  0.059099   
          2014-02-01 01:00:00                    0.065249  0.057262   
          2014-02-01 01:30:00                    0.045620  0.025670   
          2014-02-01 02:00:00                    0.030792  0.022769   

                               LightGBM_auto_stat     Theta  \
LCLid     timestamp                                           
MAC000061 2014-02-01 00:00:00            0.078041  0.020413   
          2014-02-01 00:30:00            0.057969  0.002476   
          2014-02-01 01:00:00            0.058248 -0.007650   
          2014-02-01 01:30:00            0.026564 -0.009467   
          2014-02-01 02:00:00            0.018316 -0.011248   

                               XGB Random Forest  XGB Random Forest_auto_stat  \
LCLid     timestamp                                                             
MAC000061 2014-02-01 00:00:00           0.080164                     0.065629   
          2014-02-01 00:30:00           0.061715                     0.059812   
          2014-02-01 01:00:00           0.054388                     0.050875   
          2014-02-01 01:30:00           0.044133                     0.032567   
          2014-02-01 02:00:00           0.032663                     0.029182   

                               energy_consumption  
LCLid     timestamp                                
MAC000061 2014-02-01 00:00:00               0.066  
          2014-02-01 00:30:00               0.063  
          2014-02-01 01:00:00               0.040  
          2014-02-01 01:30:00               0.020  
          2014-02-01 02:00:00               0.018

In [18]:
metrics_combined_df = pd.concat([metrics_val_df, metrics_auto_stat_val_df])
metrics_combined_df = pd.pivot(
    metrics_combined_df, index="LCLid", columns="Algorithm", values="MAE"
)
metrics_combined_df.head()

Algorithm,Lasso Regression,Lasso Regression_auto_stat,LightGBM,LightGBM_auto_stat,XGB Random Forest,XGB Random Forest_auto_stat
LCLid,,,,,,
MAC000061,0.033293,0.036640,0.030601,0.032544,0.031670,0.035716
MAC000062,0.069272,0.068969,0.074536,0.071494,0.067819,0.071182
MAC000066,0.041441,0.042819,0.039012,0.040884,0.038464,0.042638
MAC000086,0.126441,0.122261,0.103116,0.105919,0.114912,0.115007
MAC000126,0.065666,0.064992,0.064833,0.063282,0.065308,0.063359


# Combining Forecasts

In [19]:
from src.forecasting.ml_forecasting import calculate_metrics
from src.utils import ts_utils

In [20]:
def evaluate_ensemble(pred_wide, target_history, model, target, unique_id):
    metric_l = []
    for _id in tqdm(pred_wide.reset_index()[unique_id].unique()):
        # unique_mask = pred_wide[unique_id]==_id
        wide_df = pred_wide.xs(_id)
        test_target = wide_df.loc[:, target]
        y_pred = wide_df.loc[:, model]
        history = target_history.loc[target_history[unique_id] == _id, target]
        metric_l.append(
            calculate_metrics(test_target, y_pred, name=model, y_train=history)
        )
    eval_metrics_df = pd.DataFrame(metric_l)
    return {
        "Algorithm": model,
        "MAE": ts_utils.mae(
            pred_wide.loc[:, "energy_consumption"], pred_wide.loc[:, model]
        ),
        "MSE": ts_utils.mse(
            pred_wide.loc[:, "energy_consumption"], pred_wide.loc[:, model]
        ),
        "meanMASE": eval_metrics_df.loc[:, "MASE"].mean(),
        "Forecast Bias": ts_utils.forecast_bias_aggregate(
            pred_wide.loc[:, "energy_consumption"], pred_wide.loc[:, model]
        ),
    }


def highlight_abs_min(s, props=""):
    return np.where(s == np.nanmin(np.abs(s.values)), props, "")

In [21]:
def display_metrics(agg_metrics_l):
    _agg_metrics_df = pd.DataFrame(agg_metrics_l)
    display(
        _agg_metrics_df.style.format(
            {
                "MAE": "{:.4f}",
                "MSE": "{:.4f}",
                "meanMASE": "{:.4f}",
                "Forecast Bias": "{:.2f}%",
            }
        )
        .highlight_min(color="lightgreen", subset=["MAE", "MSE", "meanMASE"])
        .apply(
            highlight_abs_min,
            props="color:black;background-color:lightgreen",
            axis=0,
            subset=["Forecast Bias"],
        )
    )

In [22]:
ensemble_forecasts = [
    "FFT",
    "Lasso Regression",
    "Lasso Regression_auto_stat",
    "LightGBM",
    "LightGBM_auto_stat",
    "Theta",
    "XGB Random Forest",
    "XGB Random Forest_auto_stat",
]

In [23]:
# Picking LightGBM which is the best single model as the baseline
agg_metrics_l = agg_metrics_auto_stat_test_df.iloc[[4]].to_dict(orient="records")

## "Best-Fit"

In [24]:
# Finding the lowest metric for each LCLid
best_alg = metrics_combined_df.idxmin(axis=1)
best_alg.head()

LCLid
MAC000061              LightGBM
MAC000062     XGB Random Forest
MAC000066     XGB Random Forest
MAC000086              LightGBM
MAC000126    LightGBM_auto_stat
dtype: object

In [25]:
# Initialize two columns in the dataframe
pred_wide_test["best_fit"] = np.nan
pred_wide_test["best_fit_alg"] = ""

# For each LCL id
for lcl_id in tqdm(pred_wide_test.index.get_level_values(0).unique()):
    
    # pick the best algorithm
    alg = best_alg[lcl_id]
    
    # and store the forecast in the best_fit column
    pred_wide_test.loc[lcl_id, "best_fit"] = pred_wide_test.loc[lcl_id, alg].values
    
    # also store which model was chosen for traceability
    pred_wide_test.loc[lcl_id, "best_fit_alg"] = alg

  0%|          | 0/150 [00:00<?, ?it/s]

In [26]:
agg_metric_ = evaluate_ensemble(
    pred_wide_test, train_val_target, "best_fit", "energy_consumption", "LCLid"
)
print(agg_metric_)
agg_metrics_l.append(agg_metric_)

  0%|          | 0/150 [00:00<?, ?it/s]

{'Algorithm': 'best_fit', 'MAE': 0.07390052736481993, 'MSE': 0.026641349370239063, 'meanMASE': 0.8967544595582921, 'Forecast Bias': 0.22703696195101472}


In [27]:
display_metrics(agg_metrics_l)

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,LightGBM,0.0750,0.0268,0.9141,2.62%
1,best_fit,0.0739,0.0266,0.8968,0.23%


## Average and Median Ensemble

In [30]:
# ensemble_forecasts is a list of column names(forecast) we want to combine
pred_wide_test["average_ensemble"] = pred_wide_test[ensemble_forecasts].mean(axis=1)
pred_wide_test["median_ensemble"] = pred_wide_test[ensemble_forecasts].median(axis=1)

In [31]:
agg_metric_ = evaluate_ensemble(
    pred_wide_test, train_val_target, "median_ensemble", "energy_consumption", "LCLid"
)
print(agg_metric_)
agg_metrics_l.append(agg_metric_)
agg_metric_ = evaluate_ensemble(
    pred_wide_test, train_val_target, "average_ensemble", "energy_consumption", "LCLid"
)
print(agg_metric_)
agg_metrics_l.append(agg_metric_)

  0%|          | 0/150 [00:00<?, ?it/s]

{'Algorithm': 'median_ensemble', 'MAE': 0.07657014240155109, 'MSE': 0.027823436056040856, 'meanMASE': 0.9284026305190889, 'Forecast Bias': -0.8993317245491702}


  0%|          | 0/150 [00:00<?, ?it/s]

{'Algorithm': 'average_ensemble', 'MAE': 0.08258476472732902, 'MSE': 0.02840757624601707, 'meanMASE': 1.0111030572450495, 'Forecast Bias': 1.5005839011504623}


In [32]:
display_metrics(agg_metrics_l)

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,LightGBM,0.0750,0.0268,0.9141,2.62%
1,best_fit,0.0739,0.0266,0.8968,0.23%
2,median_ensemble,0.0766,0.0278,0.9284,-0.90%
3,average_ensemble,0.0826,0.0284,1.0111,1.50%


## Greedy Optimization

In [33]:
from src.forecasting.ensembling import calculate_performance, greedy_optimization

In [36]:
objective = partial(
    calculate_performance, pred_wide=pred_wide_val, target="energy_consumption"
)

In [41]:
solution, best_score = greedy_optimization(objective, ensemble_forecasts)

Solution: ['LightGBM', 'LightGBM_auto_stat'] | Best Score: 0.07594842304357433
Solution: ['LightGBM', 'LightGBM_auto_stat', 'Lasso Regression'] | Best Score: 0.07560408537738673
Solution cannot be improved further. Stopping optimization.


In [42]:
pred_wide_test["greedy_ensemble"] = pred_wide_test[solution].mean(axis=1)

In [43]:
agg_metric_ = evaluate_ensemble(
    pred_wide_test, train_val_target, "greedy_ensemble", "energy_consumption", "LCLid"
)
print(agg_metric_)
agg_metrics_l.append(agg_metric_)

  0%|          | 0/150 [00:00<?, ?it/s]

{'Algorithm': 'greedy_ensemble', 'MAE': 0.07327827766155419, 'MSE': 0.024935760169085986, 'meanMASE': 0.894602754919108, 'Forecast Bias': 0.8071874825882704}


In [45]:
display_metrics(agg_metrics_l)

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,LightGBM,0.0750,0.0268,0.9141,2.62%
1,best_fit,0.0739,0.0266,0.8968,0.23%
2,median_ensemble,0.0766,0.0278,0.9284,-0.90%
3,average_ensemble,0.0826,0.0284,1.0111,1.50%
4,greedy_ensemble,0.0733,0.0249,0.8946,0.81%


## Stochastic Hill-climbing with Validation Forecasts

In [46]:
from src.forecasting.ensembling import stochastic_hillclimbing

In [47]:
objective = partial(
    calculate_performance, pred_wide=pred_wide_val, target="energy_consumption"
)

In [48]:
solution, best_score = stochastic_hillclimbing(
    objective, ensemble_forecasts, n_iterations=10, init="best", random_state=42
)

Iteration: 0: Iteration did not improve the score. Solution: ['LightGBM'] | Best Score: 0.07710443567692596
Iteration: 1: Iteration did not improve the score. Solution: ['LightGBM'] | Best Score: 0.07710443567692596
Iteration: 2: Iteration did not improve the score. Solution: ['LightGBM'] | Best Score: 0.07710443567692596
Iteration: 3: Iteration did not improve the score. Solution: ['LightGBM'] | Best Score: 0.07710443567692596
Iteration: 4: Solution: ['LightGBM', 'Lasso Regression_auto_stat'] | Best Score: 0.07643731787395898
Iteration: 5: Iteration did not improve the score. Solution: ['LightGBM', 'Lasso Regression_auto_stat'] | Best Score: 0.07643731787395898
Iteration: 6: Iteration did not improve the score. Solution: ['LightGBM', 'Lasso Regression_auto_stat'] | Best Score: 0.07643731787395898
Iteration: 7: Iteration did not improve the score. Solution: ['LightGBM', 'Lasso Regression_auto_stat'] | Best Score: 0.07643731787395898
Iteration: 8: Iteration did not improve the score. So

In [49]:
pred_wide_test["stochastic_hillclimb__ensemble"] = pred_wide_test[solution].mean(axis=1)

In [50]:
agg_metric_ = evaluate_ensemble(
    pred_wide_test,
    train_val_target,
    "stochastic_hillclimb__ensemble",
    "energy_consumption",
    "LCLid",
)
print(agg_metric_)
agg_metrics_l.append(agg_metric_)

  0%|          | 0/150 [00:00<?, ?it/s]

{'Algorithm': 'stochastic_hillclimb__ensemble', 'MAE': 0.0751001803547611, 'MSE': 0.025701571800985215, 'meanMASE': 0.9203330594870778, 'Forecast Bias': 1.206903215242081}


In [51]:
display_metrics(agg_metrics_l)

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,LightGBM,0.0750,0.0268,0.9141,2.62%
1,best_fit,0.0739,0.0266,0.8968,0.23%
2,median_ensemble,0.0766,0.0278,0.9284,-0.90%
3,average_ensemble,0.0826,0.0284,1.0111,1.50%
4,greedy_ensemble,0.0733,0.0249,0.8946,0.81%
5,stochastic_hillclimb__ensemble,0.0751,0.0257,0.9203,1.21%


## Simulated Annealing with Validation Forecasts

In [52]:
from src.forecasting.ensembling import simulated_annealing

In [53]:
objective = partial(
    calculate_performance, pred_wide=pred_wide_val, target="energy_consumption"
)

In [54]:
solution, best_score = simulated_annealing(
    objective,
    ensemble_forecasts,
    p_range=(0.5, 0.0001),
    n_iterations=50,
    init="best",
    temperature_decay="geometric",
    random_state=42,
)

Finding optimum temperature range


  0%|          | 0/100 [00:00<?, ?it/s]

Iteration: 0: Solution: ['LightGBM', 'LightGBM_auto_stat'] | Best Score: 0.07594842304357433
Iteration: 1: Solution: ['LightGBM', 'LightGBM_auto_stat', 'XGB Random Forest'] | Best Score: 0.07595378259311718
Iteration: 2: Solution: ['LightGBM', 'LightGBM_auto_stat', 'XGB Random Forest', 'XGB Random Forest_auto_stat'] | Best Score: 0.07673005854842214
Iteration: 3: Iteration did not improve the score. Solution: ['LightGBM', 'LightGBM_auto_stat', 'XGB Random Forest', 'XGB Random Forest_auto_stat'] | Best Score: 0.07673005854842214
Iteration: 4: Iteration did not improve the score. Solution: ['LightGBM', 'LightGBM_auto_stat', 'XGB Random Forest', 'XGB Random Forest_auto_stat'] | Best Score: 0.07673005854842214
Iteration: 5: Solution: ['LightGBM', 'LightGBM_auto_stat', 'XGB Random Forest', 'XGB Random Forest_auto_stat', 'Lasso Regression_auto_stat'] | Best Score: 0.07641913645347752
Iteration: 6: Iteration did not improve the score. Solution: ['LightGBM', 'LightGBM_auto_stat', 'XGB Random F

In [55]:
pred_wide_test["simulated_annealing_ensemble"] = pred_wide_test[solution].mean(axis=1)

In [56]:
agg_metric_ = evaluate_ensemble(
    pred_wide_test,
    train_val_target,
    "simulated_annealing_ensemble",
    "energy_consumption",
    "LCLid",
)
print(agg_metric_)
agg_metrics_l.append(agg_metric_)

  0%|          | 0/150 [00:00<?, ?it/s]

{'Algorithm': 'simulated_annealing_ensemble', 'MAE': 0.07398726892756087, 'MSE': 0.025188022012423292, 'meanMASE': 0.9055454926772744, 'Forecast Bias': -0.41424861630617044}


In [57]:
display_metrics(agg_metrics_l)

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,LightGBM,0.0750,0.0268,0.9141,2.62%
1,best_fit,0.0739,0.0266,0.8968,0.23%
2,median_ensemble,0.0766,0.0278,0.9284,-0.90%
3,average_ensemble,0.0826,0.0284,1.0111,1.50%
4,greedy_ensemble,0.0733,0.0249,0.8946,0.81%
5,stochastic_hillclimb__ensemble,0.0751,0.0257,0.9203,1.21%
6,simulated_annealing_ensemble,0.0740,0.0252,0.9055,-0.41%


## Optimal Weighted Ensemble

In [59]:
from src.forecasting.ensembling import find_optimal_combination

In [61]:
optimal_weights = find_optimal_combination(
    ensemble_forecasts, pred_wide_val, target="energy_consumption"
)

In [62]:
pd.DataFrame({"Forecast": ensemble_forecasts, "Weights": optimal_weights}
             ).round(4).sort_values("Weights", ascending=False)

,Forecast,Weights
3,LightGBM,0.4457
4,LightGBM_auto_stat,0.2781
2,Lasso Regression_auto_stat,0.1381
1,Lasso Regression,0.0841
6,XGB Random Forest,0.0541
0,FFT,0.0000
5,Theta,0.0000
7,XGB Random Forest_auto_stat,0.0000


In [63]:
pred_wide_test["optimal_combination_ensemble"] = np.sum(
    pred_wide_test[ensemble_forecasts].values * np.array(optimal_weights), axis=1
)

In [64]:
agg_metric_ = evaluate_ensemble(
    pred_wide_test,
    train_val_target,
    "optimal_combination_ensemble",
    "energy_consumption",
    "LCLid",
)
print(agg_metric_)
agg_metrics_l.append(agg_metric_)

  0%|          | 0/150 [00:00<?, ?it/s]

{'Algorithm': 'optimal_combination_ensemble', 'MAE': 0.07304274088138435, 'MSE': 0.024662486872771932, 'meanMASE': 0.8939078585943616, 'Forecast Bias': 0.8487786644069247}


In [65]:
display_metrics(agg_metrics_l)

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,LightGBM,0.0750,0.0268,0.9141,2.62%
1,best_fit,0.0739,0.0266,0.8968,0.23%
2,median_ensemble,0.0766,0.0278,0.9284,-0.90%
3,average_ensemble,0.0826,0.0284,1.0111,1.50%
4,greedy_ensemble,0.0733,0.0249,0.8946,0.81%
5,stochastic_hillclimb__ensemble,0.0751,0.0257,0.9203,1.21%
6,simulated_annealing_ensemble,0.0740,0.0252,0.9055,-0.41%
7,optimal_combination_ensemble,0.0730,0.0247,0.8939,0.85%


## Stacking/Blending Model

In [66]:
from sklearn.linear_model import (
    HuberRegressor,
    LassoCV,
    LinearRegression,
    RidgeCV
)

### Linear Regression

In [67]:
stacking_model = LinearRegression(positive=True, fit_intercept=False)
stacking_model.fit(
    pred_wide_val[ensemble_forecasts], pred_wide_val["energy_consumption"]
)

LinearRegression(fit_intercept=False, positive=True)

In [68]:
pd.DataFrame({"Forecast": ensemble_forecasts, "Weights": stacking_model.coef_}
             ).round(4).sort_values("Weights", ascending=False)

,Forecast,Weights
3,LightGBM,0.4180
2,Lasso Regression_auto_stat,0.2715
1,Lasso Regression,0.2162
4,LightGBM_auto_stat,0.1268
0,FFT,0.0000
5,Theta,0.0000
6,XGB Random Forest,0.0000
7,XGB Random Forest_auto_stat,0.0000


In [69]:
pred_wide_test["linear_reg_blending"] = stacking_model.predict(
    pred_wide_test[ensemble_forecasts]
)

In [70]:
agg_metric_ = evaluate_ensemble(
    pred_wide_test,
    train_val_target,
    "linear_reg_blending",
    "energy_consumption",
    "LCLid",
)
print(agg_metric_)
agg_metrics_l.append(agg_metric_)

  0%|          | 0/150 [00:00<?, ?it/s]

{'Algorithm': 'linear_reg_blending', 'MAE': 0.07548698510696149, 'MSE': 0.02446766144238813, 'meanMASE': 0.9255525743936289, 'Forecast Bias': 4.357424212410399}


In [71]:
display_metrics(agg_metrics_l)

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,LightGBM,0.0750,0.0268,0.9141,2.62%
1,best_fit,0.0739,0.0266,0.8968,0.23%
2,median_ensemble,0.0766,0.0278,0.9284,-0.90%
3,average_ensemble,0.0826,0.0284,1.0111,1.50%
4,greedy_ensemble,0.0733,0.0249,0.8946,0.81%
5,stochastic_hillclimb__ensemble,0.0751,0.0257,0.9203,1.21%
6,simulated_annealing_ensemble,0.0740,0.0252,0.9055,-0.41%
7,optimal_combination_ensemble,0.0730,0.0247,0.8939,0.85%
8,linear_reg_blending,0.0755,0.0245,0.9256,4.36%


### Ridge Regression

In [72]:
stacking_model = RidgeCV()
stacking_model.fit(
    pred_wide_val[ensemble_forecasts], pred_wide_val["energy_consumption"]
)

RidgeCV()

In [73]:
pd.DataFrame({"Forecast": ensemble_forecasts, "Weights": stacking_model.coef_}).round(
    4
).sort_values("Weights", ascending=False)

,Forecast,Weights
3,LightGBM,0.4724
1,Lasso Regression,0.3335
4,LightGBM_auto_stat,0.2556
2,Lasso Regression_auto_stat,0.2336
0,FFT,-0.0225
5,Theta,-0.0271
7,XGB Random Forest_auto_stat,-0.0732
6,XGB Random Forest,-0.1599


In [74]:
pred_wide_test["ridge_reg_blending"] = stacking_model.predict(
    pred_wide_test[ensemble_forecasts]
)

In [75]:
agg_metric_ = evaluate_ensemble(
    pred_wide_test,
    train_val_target,
    "ridge_reg_blending",
    "energy_consumption",
    "LCLid",
)
print(agg_metric_)
agg_metrics_l.append(agg_metric_)

  0%|          | 0/150 [00:00<?, ?it/s]

{'Algorithm': 'ridge_reg_blending', 'MAE': 0.07366254445513652, 'MSE': 0.024282906903738806, 'meanMASE': 0.9074968710140813, 'Forecast Bias': 1.956933465529595}


In [76]:
display_metrics(agg_metrics_l)

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,LightGBM,0.0750,0.0268,0.9141,2.62%
1,best_fit,0.0739,0.0266,0.8968,0.23%
2,median_ensemble,0.0766,0.0278,0.9284,-0.90%
3,average_ensemble,0.0826,0.0284,1.0111,1.50%
4,greedy_ensemble,0.0733,0.0249,0.8946,0.81%
5,stochastic_hillclimb__ensemble,0.0751,0.0257,0.9203,1.21%
6,simulated_annealing_ensemble,0.0740,0.0252,0.9055,-0.41%
7,optimal_combination_ensemble,0.0730,0.0247,0.8939,0.85%
8,linear_reg_blending,0.0755,0.0245,0.9256,4.36%
9,ridge_reg_blending,0.0737,0.0243,0.9075,1.96%


In [57]:
# ts_utils.mae(pred_wide_val['energy_consumption'], stacking_model.predict(pred_wide_val[ensemble_forecasts]))

# ts_utils.mae(pred_wide_test['energy_consumption'], stacking_model.predict(pred_wide_test[ensemble_forecasts]))

### Lasso Regression

In [77]:
stacking_model = LassoCV()
stacking_model.fit(
    pred_wide_val[ensemble_forecasts], pred_wide_val["energy_consumption"]
)

LassoCV()

In [78]:
pd.DataFrame({"Forecast": ensemble_forecasts, "Weights": stacking_model.coef_}).round(
    4
).sort_values("Weights", ascending=False)

,Forecast,Weights
3,LightGBM,0.4533
1,Lasso Regression,0.3015
2,Lasso Regression_auto_stat,0.2436
4,LightGBM_auto_stat,0.2203
0,FFT,-0.0187
5,Theta,-0.0257
7,XGB Random Forest_auto_stat,-0.0486
6,XGB Random Forest,-0.1084


In [79]:
pred_wide_test["lasso_reg_blending"] = stacking_model.predict(
    pred_wide_test[ensemble_forecasts]
)

In [80]:
agg_metric_ = evaluate_ensemble(
    pred_wide_test,
    train_val_target,
    "lasso_reg_blending",
    "energy_consumption",
    "LCLid",
)
print(agg_metric_)
agg_metrics_l.append(agg_metric_)

  0%|          | 0/150 [00:00<?, ?it/s]

{'Algorithm': 'lasso_reg_blending', 'MAE': 0.07363675737967389, 'MSE': 0.024275200438200837, 'meanMASE': 0.9070298259319937, 'Forecast Bias': 2.033906907644201}


In [81]:
display_metrics(agg_metrics_l)

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,LightGBM,0.0750,0.0268,0.9141,2.62%
1,best_fit,0.0739,0.0266,0.8968,0.23%
2,median_ensemble,0.0766,0.0278,0.9284,-0.90%
3,average_ensemble,0.0826,0.0284,1.0111,1.50%
4,greedy_ensemble,0.0733,0.0249,0.8946,0.81%
5,stochastic_hillclimb__ensemble,0.0751,0.0257,0.9203,1.21%
6,simulated_annealing_ensemble,0.0740,0.0252,0.9055,-0.41%
7,optimal_combination_ensemble,0.0730,0.0247,0.8939,0.85%
8,linear_reg_blending,0.0755,0.0245,0.9256,4.36%
9,ridge_reg_blending,0.0737,0.0243,0.9075,1.96%


### Huber Regression

To optimize MAE, we can also use Huber Regressor which uses Huber Loss

In [82]:
stacking_model = HuberRegressor()
stacking_model.fit(
    pred_wide_val[ensemble_forecasts], pred_wide_val["energy_consumption"]
)

HuberRegressor()

In [83]:
pd.DataFrame({"Forecast": ensemble_forecasts, "Weights": stacking_model.coef_}).round(
    4
).sort_values("Weights", ascending=False)

,Forecast,Weights
3,LightGBM,0.4013
4,LightGBM_auto_stat,0.2743
1,Lasso Regression,0.1806
2,Lasso Regression_auto_stat,0.1511
6,XGB Random Forest,0.1090
5,Theta,-0.0181
0,FFT,-0.0538
7,XGB Random Forest_auto_stat,-0.0881


In [84]:
pred_wide_test["huber_reg_blending"] = stacking_model.predict(
    pred_wide_test[ensemble_forecasts]
)

In [85]:
agg_metric_ = evaluate_ensemble(
    pred_wide_test,
    train_val_target,
    "huber_reg_blending",
    "energy_consumption",
    "LCLid",
)
print(agg_metric_)
agg_metrics_l.append(agg_metric_)

  0%|          | 0/150 [00:00<?, ?it/s]

{'Algorithm': 'huber_reg_blending', 'MAE': 0.07033893874148929, 'MSE': 0.024571137170415005, 'meanMASE': 0.8969600071808124, 'Forecast Bias': -6.474402288985733}


In [86]:
display_metrics(agg_metrics_l)

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,LightGBM,0.0750,0.0268,0.9141,2.62%
1,best_fit,0.0739,0.0266,0.8968,0.23%
2,median_ensemble,0.0766,0.0278,0.9284,-0.90%
3,average_ensemble,0.0826,0.0284,1.0111,1.50%
4,greedy_ensemble,0.0733,0.0249,0.8946,0.81%
5,stochastic_hillclimb__ensemble,0.0751,0.0257,0.9203,1.21%
6,simulated_annealing_ensemble,0.0740,0.0252,0.9055,-0.41%
7,optimal_combination_ensemble,0.0730,0.0247,0.8939,0.85%
8,linear_reg_blending,0.0755,0.0245,0.9256,4.36%
9,ridge_reg_blending,0.0737,0.0243,0.9075,1.96%


## Bonus: Regularization through Variety

### Correlation as Variety

In [87]:
from src.utils.plotting_utils import plot_correlation_plot

In [88]:
fig = plot_correlation_plot(
    pred_wide_val[ensemble_forecasts].corr(),
    title="Correlation of the forecasts",
    figsize=(800, 800),
)
fig.write_image("imgs/chapter_8/correlation.png")
fig.show()

### Using Variety as Regularization

In [89]:
from src.forecasting.ensembling import calculate_diversity

In [90]:
def calculate_diverse_objective(ens, pred_wide, target, diversity_matrix, alpha):
    perf = calculate_performance(ens, pred_wide, target)
    div = calculate_diversity(ens, diversity_matrix)
    return perf + alpha * div

In [91]:
objective = partial(
    calculate_diverse_objective,
    pred_wide=pred_wide_val,
    target="energy_consumption",
    diversity_matrix=pred_wide_val[ensemble_forecasts].corr(),
    alpha=0.05,
)

In [92]:
solution, best_score = stochastic_hillclimbing(
    objective, ensemble_forecasts, n_iterations=10, random_state=42
)

Iteration: 0: Solution: ['LightGBM', 'XGB Random Forest'] | Best Score: 0.12639151168634374
Iteration: 1: Iteration did not improve the score. Solution: ['LightGBM', 'XGB Random Forest'] | Best Score: 0.12639151168634374
Iteration: 2: Iteration did not improve the score. Solution: ['LightGBM', 'XGB Random Forest'] | Best Score: 0.12639151168634374
Iteration: 3: Solution: ['LightGBM', 'XGB Random Forest', 'XGB Random Forest_auto_stat'] | Best Score: 0.1256250266997805
Iteration: 4: Solution: ['LightGBM', 'XGB Random Forest', 'XGB Random Forest_auto_stat', 'Lasso Regression_auto_stat'] | Best Score: 0.12407977957273272
Iteration: 5: Iteration did not improve the score. Solution: ['LightGBM', 'XGB Random Forest', 'XGB Random Forest_auto_stat', 'Lasso Regression_auto_stat'] | Best Score: 0.12407977957273272
Iteration: 6: Iteration did not improve the score. Solution: ['LightGBM', 'XGB Random Forest', 'XGB Random Forest_auto_stat', 'Lasso Regression_auto_stat'] | Best Score: 0.1240797795727

In [74]:
# ts_utils.mae(pred_wide_test['energy_consumption'], pred_wide_test[solution].mean(axis=1).values)

In [93]:
pred_wide_test["hillclimbing_w_reg_ensemble"] = pred_wide_test[solution].mean(axis=1)

In [94]:
agg_metric_ = evaluate_ensemble(
    pred_wide_test,
    train_val_target,
    "hillclimbing_w_reg_ensemble",
    "energy_consumption",
    "LCLid",
)
print(agg_metric_)
agg_metrics_l.append(agg_metric_)

  0%|          | 0/150 [00:00<?, ?it/s]

{'Algorithm': 'hillclimbing_w_reg_ensemble', 'MAE': 0.07467400714416188, 'MSE': 0.02567434743870161, 'meanMASE': 0.9139986417373478, 'Forecast Bias': -0.5722344991439777}


In [95]:
display_metrics(agg_metrics_l)

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,LightGBM,0.0750,0.0268,0.9141,2.62%
1,best_fit,0.0739,0.0266,0.8968,0.23%
2,median_ensemble,0.0766,0.0278,0.9284,-0.90%
3,average_ensemble,0.0826,0.0284,1.0111,1.50%
4,greedy_ensemble,0.0733,0.0249,0.8946,0.81%
5,stochastic_hillclimb__ensemble,0.0751,0.0257,0.9203,1.21%
6,simulated_annealing_ensemble,0.0740,0.0252,0.9055,-0.41%
7,optimal_combination_ensemble,0.0730,0.0247,0.8939,0.85%
8,linear_reg_blending,0.0755,0.0245,0.9256,4.36%
9,ridge_reg_blending,0.0737,0.0243,0.9075,1.96%
